In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain.llms import OpenAI

## Simple LLM inference


In [ ]:
# Temperature measures randomness
# Make sure OPENAI_API_KEY environmental variable is set
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

In [ ]:
text = "Suggest a suitable meal plan for diabetes patients"
print(llm(text))

## LLM Chains

A chain generally has an llm, prompt and optionally an output parset


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

prompt = PromptTemplate(
    input_variables=["power"],
    template="A good name for a superhero who can {power}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("Go through walls"))

## Incorporating memory


In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    # Type of memory to use in the conversation chain
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)

## Incorporated vector stores (deeplake)


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA


# instantiate the LLM and embeddings models
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# Embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
  "World War II, spanning from 1939 to 1945, was the deadliest conflict in human history, involving over 30 countries and resulting in approximately 70-85 million fatalities.",
  "It was sparked by the aggressive expansionism of Nazi Germany under Adolf Hitler and imperial Japan's quest for dominance in Asia.",
  "The war witnessed the widespread use of new technologies such as tanks, aircraft, and atomic bombs, leading to unprecedented devastation and loss of life. Key events include the invasion of Poland, the Battle of Stalingrad, the Normandy landings, and the atomic bombings of Hiroshima and Nagasaki." 
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# create Deep Lake dataset
my_activeloop_org_id = "rashadism" 
my_activeloop_dataset_name = "langchain101"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

## Create a RetrievalQA chain

In [ ]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

## An agent that uses RetrievalQA as a tool

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [ ]:
response = agent.run("What was world war 2 sparked by?")
print(response)

In [ ]:
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
    "During World War II, the German military used an encryption device called the Enigma machine to encode their messages. The British mathematician Alan Turing played a key role in breaking the Enigma code, leading to significant advancements in code-breaking and intelligence gathering.",
    "The Manhattan Project was a top-secret research and development project during World War II that produced the first nuclear weapons. Led by the United States with contributions from the United Kingdom and Canada, the project culminated in the bombings of Hiroshima and Nagasaki in August 1945, marking the only wartime use of nuclear weapons in history."
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	# verbose=True
)

In [ ]:
response = agent.run("What did Alan Turing break?")
print(response)

## An agent that searches Google

In [ ]:
from langchain.llms import OpenAI

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

In [ ]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

In [ ]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

In [ ]:
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)

In [ ]:
response = agent("Who is the current partner of DiCaprio?")
print(response['output'])

## An agent with multiple tools
Here, the agent is provided with more than one tool, and the agent decides which tool to use based on the context

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [ ]:
# Create an LLM chain for text summarization

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
# Create an LLM chain for writing a poem

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a poem about the following: {query}"
)

poem_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    ),
    Tool(
       name='Poem',
       func=poem_chain.run,
       description='useful to write a poem on a given topic'
    )
]

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

In [ ]:
search_response = agent("Which player is the leading wicket taker for Mumbai Indians? (MI)")
print(search_response['output'])